In [ ]:
%load_ext autoreload
%autoreload 2

# Processing each task
from numpy import ndarray
from pandas import DataFrame, read_csv
from matplotlib.pyplot import savefig, show, figure
from pathlib import Path
import sys

# Add the project directory to sys.path
project_dir = Path.cwd().parent.parent  # Adjust as needed to point to your project root

# Import the module
from utils.dslabs_functions import get_variable_types, encode_cyclic_variables, dummify
from utils.data_loader import DataLoader


dataloader = DataLoader()
data, target = dataloader.get_security_classification_dataset_and_target()
# data: DataFrame = read_csv("data/stroke_mvi.csv", index_col="id", na_values="")
vars: dict[str, list] = get_variable_types(data)

yes_no: dict[str, int] = {"no": 0, "No": 0, "yes": 1, "Yes": 1}
residence_type_values: dict[str, int] = {"Rural": 0, "Urban": 1}

encoding: dict[str, dict[str, int]] = {
    # "LAW_CAT_CD": residence_type_values,
    # "hypertension": yes_no,
}
df: DataFrame = data.replace(encoding, inplace=False)
df.head()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


,ARREST_KEY,ARREST_DATE,PD_CD,PD_DESC,KY_CD,OFNS_DESC,LAW_CODE,LAW_CAT_CD,ARREST_BORO,ARREST_PRECINCT,JURISDICTION_CODE,AGE_GROUP,PERP_SEX,PERP_RACE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude
0,32311380,2007-06-18,511.0,"CONTROLLED SUBSTANCE, POSSESSION 7",235.0,DANGEROUS DRUGS,PL 2200300,M,Q,27,1.0,18-24,M,BLACK,NaN,NaN,NaN,NaN
1,192799737,2019-01-26,177.0,SEXUAL ABUSE,116.0,SEX CRIMES,PL 1306503,F,M,25,0.0,45-64,M,BLACK,1000555.0,230994.0,40.800694,-73.941109
2,193260691,2019-02-06,NaN,NaN,NaN,NaN,PL 2203400,F,M,14,0.0,25-44,M,UNKNOWN,986685.0,215375.0,40.757839,-73.991212
3,149117452,2016-01-06,153.0,RAPE 3,104.0,RAPE,PL 1302503,F,K,67,0.0,25-44,M,BLACK,998032.0,175598.0,40.648650,-73.950336
4,190049060,2018-11-15,157.0,RAPE 1,104.0,RAPE,PL 1303501,F,K,77,0.0,25-44,M,BLACK,1003606.0,185050.0,40.674583,-73.930222


In [10]:

for v in vars["symbolic"]:
    print(v, data[v].unique())

PD_DESC ['CONTROLLED SUBSTANCE, POSSESSION 7' 'SEXUAL ABUSE' nan 'RAPE 3' 'RAPE 1'
 'TRESPASS 3, CRIMINAL' 'SEXUAL ABUSE 3,2' 'ASSAULT 3' 'STRANGULATION 1ST'
 'PROSTITUTION' 'SEXUAL ABUSE 1' 'SODOMY 1' 'WEAPONS POSSESSION 3'
 'RIOT 1' 'MARIJUANA, POSSESSION 1, 2 & 3'
 'IMPERSONATION 2, PUBLIC SERVANT' 'SODOMY 3'
 'STOLEN PROPERTY 3,POSSESSION' 'MARIJUANA, SALE 4 & 5'
 'RECKLESS ENDANGERMENT 1' 'FORGERY,ETC.,UNCLASSIFIED-FELONY'
 'MARIJUANA, POSSESSION 4 & 5' 'BURGLARY,RESIDENCE,NIGHT'
 'WEAPONS POSSESSION 1 & 2' 'TRESPASS 2, CRIMINAL' 'RAPE 2'
 'ROBBERY,UNCLASSIFIED,OPEN AREAS' 'FORGERY,ETC.-MISD.' 'ARSON 2,3,4'
 'ASSAULT 2,1,UNCLASSIFIED' 'WEAPONS, POSSESSION, ETC'
 'IMPERSONATION 2, PUBLIC SERVAN' 'FORGERY,ETC.,UNCLASSIFIED-FELO'
 'IDENTITY THFT-1' 'BURGLARY,COMMERCIAL,DAY' 'AGGRAVATED HARASSMENT 2'
 'ROBBERY,OPEN AREA UNCLASSIFIED' 'BURGLARY,UNCLASSIFIED,UNKNOWN'
 'BURGLARY,UNCLASSIFIED,UNKNOWN TIME' 'CONTROLLED SUBSTANCE,SALE 3'
 'CONTROLLED SUBSTANCE,INTENT TO SELL 3' 'RECKLESS EN

In [ ]:
from math import pi, sin, cos
from numpy import ndarray
from pandas import DataFrame, read_csv, concat
from sklearn.preprocessing import OneHotEncoder


def encode_cyclic_variables(data: DataFrame, vars: list[str]) -> None:
    for v in vars:
        x_max: float | int = max(data[v])
        data[v + "_sin"] = data[v].apply(lambda x: round(sin(2 * pi * x / x_max), 3))
        data[v + "_cos"] = data[v].apply(lambda x: round(cos(2 * pi * x / x_max), 3))
    return


def dummify(df: DataFrame, vars_to_dummify: list[str]) -> DataFrame:
    other_vars: list[str] = [c for c in df.columns if not c in vars_to_dummify]

    enc = OneHotEncoder(
        handle_unknown="ignore", sparse_output=False, dtype="bool", drop="if_binary"
    )
    trans: ndarray = enc.fit_transform(df[vars_to_dummify])

    new_vars: ndarray = enc.get_feature_names_out(vars_to_dummify)
    dummy = DataFrame(trans, columns=new_vars, index=df.index)

    final_df: DataFrame = concat([df[other_vars], dummy], axis=1)
    return final_df


In [ ]:
from sklearn.discriminant_analysis import StandardScaler
df = dummify(data, vars["symbolic"])


KeyError: 'numerical'

In [15]:
# Scale numerical variables
scaler = StandardScaler()
df[vars["numeric"]] = scaler.fit_transform(df[vars["numeric"]])

In [17]:
# Age group mapping
age_map = {"18-24": 1, "25-44": 2, "45-64": 3, "65+": 4, "<18": 0}
df["AGE_GROUP"] = data["AGE_GROUP"].map(age_map).fillna(-1)

# Encode race
race_encoder = {race: idx for idx, race in enumerate(data["PERP_RACE"].unique())}
df["PERP_RACE"] = data["PERP_RACE"].map(race_encoder)

df.head()

,ARREST_KEY,ARREST_DATE,PD_CD,KY_CD,LAW_CAT_CD,ARREST_PRECINCT,JURISDICTION_CODE,PERP_SEX,X_COORD_CD,Y_COORD_CD,...,PERP_RACE_AMERICAN INDIAN/ALASKAN NATIVE,PERP_RACE_ASIAN / PACIFIC ISLANDER,PERP_RACE_BLACK,PERP_RACE_BLACK HISPANIC,PERP_RACE_OTHER,PERP_RACE_UNKNOWN,PERP_RACE_WHITE,PERP_RACE_WHITE HISPANIC,AGE_GROUP,PERP_RACE
0,-1.209148,2007-06-18,0.089251,-0.239799,M,-0.989799,-0.033800,M,NaN,NaN,...,False,False,True,False,False,False,False,False,1.0,0
1,1.501840,2019-01-26,-1.203194,-1.034823,F,-1.048334,-0.137549,M,-0.241446,0.099981,...,False,False,True,False,False,False,False,False,3.0,0
2,1.509627,2019-02-06,NaN,NaN,F,-1.370277,-0.137549,M,-0.928376,0.002333,...,False,False,False,False,False,True,False,False,2.0,1
3,0.763954,2016-01-06,-1.296064,-1.114993,F,0.180900,-0.137549,M,-0.366401,-0.246347,...,False,False,True,False,False,False,False,False,2.0,0
4,1.455376,2018-11-15,-1.280586,-1.114993,F,0.473575,-0.137549,M,-0.090341,-0.187255,...,False,False,True,False,False,False,False,False,2.0,0
